In [ ]:
import pandas as pd
from collections import Counter

In [ ]:
filepath="/home/mitsuki/data/pnnl/massive.ucsd.edu/result/Arthrobacter_sp_FB24/Arthro_001-LTQFT1_28Sep05_Andro_0705-06_msgfplus.tsv"
msgf_df=pd.read_csv(filepath,sep='\t')
fmsgf_df=msgf_df[msgf_df["PepQValue"]<=0.05]

In [ ]:
fmsgf_df.shape

In [ ]:
filepath="/home/mitsuki/out/altorf/proteome/result/Arthrobacter_sp_FB24/Arthro_001-LTQFT1_28Sep05_Andro_0705-06_annotated.tsv"
df=pd.read_csv(filepath,sep='\t')
f_df=df[df["PepQValue"]<=0.05]
f_df.head()

In [ ]:
f_df.shape

In [ ]:
def judge_origin(protein):
    if protein[:3].upper()=='REF' or protein[:2]=="gb" or protein[:2]=="gi":
        return 'REF'
    elif protein[:3]=="XXX":
        return "XXX"
    else:
        return "CON"

In [ ]:
origin_lst=[judge_origin(i) for i in fmsgf_df["Protein"]]
counter=Counter(origin_lst)
print(counter)

In [ ]:
origin_lst=[judge_origin(i) for i in f_df["Protein"]]
counter=Counter(origin_lst)
print(counter)

In [ ]:
df.head()

In [ ]:
filename="/home/mitsuki/altorf/proteome/createcatalog/pnnl_downloads.csv"
catalogDf=pd.read_csv(filename)

dct_lst=[]
for key, row in catalogDf.iterrows():
    org=row["Organism"]
    data=row["Dataset"]
    
    fastaType_lst=["msgfplus","annotated"]
    dir_lst=["/home/mitsuki/data/pnnl/massive.ucsd.edu/result/"+org+"/",
             "/home/mitsuki/out/altorf/proteome/result/"+org+"/"]
    filepath_lst=(dir_lst[0]+data+"_msgfplus.tsv",
                  dir_lst[1]+data+"_annotated.tsv")
    
    counter_lst=[]
    for filepath in filepath_lst:
        df=pd.read_csv(filepath,sep='\t')
        fdf=df[df["PepQValue"]<=0.05]#filter by peptide level FDR 0.05
        origin_lst=[judge_origin(i) for i in fdf["Protein"]]
        counter=Counter(origin_lst)
        counter_lst.append(counter)
        
    for i,fastaType in enumerate(fastaType_lst):
        dct={}
        dct["Organism"]=org
        dct["Dataset"]=data
        dct["fasta_type"]=fastaType
        dct["REF"]=counter_lst[i]["REF"]
        dct["CON"]=counter_lst[i]["CON"]
        dct["XXX"]=counter_lst[i]["XXX"]
        dct_lst.append(dct)
        
    print("DONE "+org+'/'+data)

In [ ]:
counter_lst

In [ ]:
dct_lst=[]
for i,fastaType in enumerate(fastaType_lst):
    dct={}
    dct["Organism"]=org
    dct["Dataset"]=data
    dct["fasta_type"]=fastaType
    dct["REF"]=counter_lst[i]["REF"]
    dct["CON"]=counter_lst[i]["CON"]
    dct["XXX"]=counter_lst[i]["XXX"]
    dct_lst.append(dct)

In [ ]:
dct_lst

In [ ]:
out_df=pd.DataFrame(dct_lst)
out_df=out_df[["Organism","Dataset","fasta_type","REF","CON","XXX"]]
out_df.tail(10)